In [48]:
import time
import logging
from pathlib import Path
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.coordinates import Distance
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from utils import (
    get_source_identifier,
    insert_space_source_ids,
    remove_space_source_ids,
    convert_F_nu_to_luminosity,
)


log = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s|%(name)s|%(message)s",
    datefmt="%m/%d/%Y %I:%M:%S %p",
)

In [49]:
def find_xmatch_id_in_catalog(
    xmatch_id, xmatch_id_column, catalog, catalog_id_column, id_strip=None
):
    """Given a catalogue with cross-matched sources, return the sources
    corresponding to a given counterpart."""
    if id_strip is None:
        this_source = catalog[xmatch_id_column] == xmatch_id
    else:
        if catalog[xmatch_id_column].dtype == np.int64:
            this_source = catalog[xmatch_id_column] == np.int64(
                xmatch_id.strip(id_strip)
            )
        else:
            this_source = catalog[xmatch_id_column] == xmatch_id.strip(id_strip)

    xmatches = catalog[this_source]

    if len(xmatches) == 0:
        log.info(f"{xmatch_id} not matched in the catalogue")
    else:
        log.info(f"{xmatch_id} matched with {xmatches[catalog_id_column].data}")
        return xmatches

def convert_flux_to_luminosity(flux, flux_unit, distance):
    
    flux = flux * u.Unit(flux_unit)
    D_L = distance.to(u.cm) # Convert Mpc to cm for consistent units
    flux_in_cgs = flux.to(u.erg / (u.s * u.cm**2))
    luminosity = 4 * np.pi * D_L**2 * flux_in_cgs
    
    return luminosity.to(u.erg / u.s)

# Load the CoreG and FR0 catalogues

In [52]:
#CoreG Catalogs
ho_1997 = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/ApJS/112/315")
nagar_2005 = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/A+A/435/521")
#FR0 Catalog
fr0cat = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/A+A/609/A1")
#MORX Catalog
morx = Vizier(columns=["**"], row_limit=-1).get_catalogs("V/158/morxv2")
torresi_sources = [
    "SDSS J004150.47−091811.2",
    "SDSS J010101.12−002444.4",
    "SDSS J011515.78+001248.4",
    "SDSS J015127.10−083019.3",
    "SDSS J080624.94+172503.7",
    "SDSS J092405.30+141021.5",
    "SDSS J093346.08+100909.0",
    "SDSS J094319.15+361452.1",
    "SDSS J104028.37+091057.1",
    "SDSS J114232.84+262919.9",
    "SDSS J115954.66+302726.9",
    "SDSS J122206.54+134455.9",
    "SDSS J125431.43+262040.6",
    "Tol 1326−379",
    "SDSS J135908.74+280121.3",
    "SDSS J153901.66+353046.0",
    "SDSS J160426.51+174431.1",
    "SDSS J171522.97+572440.2",
    "SDSS J235744.10−001029.9",
]
#Swift-BAT Catalogs of 105 month survey, Detected AGN in 60 months, 
swift_bat_105_months = Vizier(columns=["**"], row_limit=-1).get_catalogs("J/ApJS/235/4")

09/03/2025 09:02:44 AM|astroquery|UnitsWarning: Unit 'pct' not supported by the VOUnit standard. Did you mean PC, PT, Pct, pC, pT or pc?


In [53]:
morx[0]

recno,RAJ2000,DEJ2000,Name,Type,Rmag,Bmag,Comment,R,B,z,Cite,Zcite,RXpct,QSOpct,galpct,starpct,Xraypct,CX-ID,XMM-ID,Swift-ID,OtherXID,radiopct,VLASS-ID,FIRST-ID,LoTSS-ID,RACS-ID,NVSS-ID,Lobe1,Lobe2,Lobedist
,deg,deg,,,mag,mag,,,,,,,pct,pct,pct,pct,pct,,,,,pct,,,,,,,,arcsec
int32,float64,float64,str25,str4,float32,float32,str3,str1,str1,float32,str6,str6,int16,int16,int16,int16,int16,str22,str22,str22,str22,int16,str22,str22,str22,str22,str22,str22,str22,int16
1,0.0001707,27.7728782,J000000.04+274622.3,R,21.47,22.44,gG,-,-,--,MQ,,94,71,22,1,--,,,,,94,,,ILTJ240000.00+274622.1,,,,,--
2,0.0002102,37.1905960,J000000.05+371126.1,R,19.75,20.54,jN,-,1,--,MQ,,79,50,29,0,--,,,,,79,,,ILTJ000000.20+371125.6,,,,,--
3,0.0003456,-6.4919398,SDSS J000000.08-062930.9,GR,21.75,23.20,g,1,1,0.514,DR16,DR16,98,5,93,0,--,,,,,98,VLAJ000000.07-062930.6,,,,,,,--
4,0.0003539,32.1366100,PGC 1982072,GR,15.47,16.43,p+,1,1,--,PGC,,99,0,99,0,--,,,,,99,,,ILTJ000000.08+320812.2,,,,,--
5,0.0004167,33.9321723,J000000.10+335555.8,R,20.08,21.47,pN,1,-,0.300,MQ,MQ,96,49,46,1,--,,,,,96,,,ILTJ000000.17+335555.6,,,,,--
6,0.0006461,-45.9542473,PGC 520795,GR,13.78,14.20,j+G,1,1,--,PGC,,91,0,89,2,--,,,,,91,,SUMSSJ000000.1-455721,,RACS J000000.0-455714,,,,--
7,0.0006575,-20.0743440,UVQSJ000000.15-200427.7,AX,17.90,18.46,pG,-,1,0.291,UVQS,UVQS,99,95,4,0,99,,,LSXPSJ000000.1-200426,,--,,,,,,,,--
8,0.0008461,26.2650693,J000000.20+261554.2,R,22.14,23.51,gN,1,1,--,MQ,,96,8,88,0,--,,,,,96,,,ILTJ000000.26+261554.8,,,,,--


In [64]:
catalogue = Table(
    names=(
        "SOURCE_NAME",
        "SOURCE_TYPE",
        "SDSS-ID",
        "FERMI-ID",
        "NVSS-XMATCH-ID",
        "NVSS-MORX-ID",
        "FIRST-XMATCH-ID",
        "FIRST-MORX-ID",
        "XMM-MORX-ID",
        "CXO-MORX-ID",
        "SWIFT-MORX-ID",
        "LoTSS-MORX-ID",
        "VLASS-MORX",
        "LOBE EXTENSION",
        "DISTANCE",
        "TORESSI DETECTION",
        "NVSS-FLUX-XMATCH",
        "NVSS-FLUX-ERROR-XMATCH",
        "FIRST-FLUX-XMATCH",
        "FIRST-FLUX-ERROR-XMATCH",
    ),
    dtype=[
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        str,
        np.float64,
        np.float64,
        bool,
        np.float64,
        np.float64,
        np.float64,
        np.float64
    ],
    units=[
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "mas",
        "Mpc",
        "",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        "erg s-1",
        
    ],
)

In [65]:
first_xmatches = XMatch.query(
    cat1=nagar_2005[0],
    cat2="vizier:VIII/92/first14",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=10 * u.arcsec,
)

first_xmatches

angDist,recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE,_RAJ2000,_DEJ2000,FIRST,FITS,RAJ2000,DEJ2000,p(S),Fpeak,Fint,Rms,Maj,Min,PA,N1,c1,N2,Ep,Ep(JD),s_Ep(JD)
arcsec,,,,,,hms,dms,,,,,,,,,,,,deg,deg,,,"""h:m:s""","""d:m:s""",,mJy,mJy,mJy,arcsec,arcsec,deg,,,,yr,d,d
float64,int16,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float32,float64,float64,str16,str4,str12,str12,float32,float64,float64,float32,float32,float32,float32,int16,str1,int16,float64,float64,float64
0.944799,14,NGC524,T2:,-1.0,32.1,,,--,1.5,,1.5,,20.27,3,Simbad,NED,21.1992,9.5389,21.198963,9.538781,J012447.7+093219,FITS,01 24 47.751,+09 32 19.61,0.014,1.11,0.72,0.132,0.96,0.0,90.2,2,g,2,2009.378,2454970.4,0.348
5.977018,20,NGC1055,T2/L2::,3.0,12.6,,,--,--,<,1.8,<,19.53,0,Simbad,NED,40.4387,0.4423,40.438246,0.443897,J024145.1+002638,FITS,02 41 45.179,+00 26 38.03,0.014,6.91,11.9,0.096,5.91,3.81,94.5,0,,3,1998.142,2450866.2,1174.939
2.125878,30,NGC2681,L1.9,0.0,13.3,,,--,--,<,1.4,<,19.48,0,Simbad,NED,133.386,51.3132,133.386462,51.313717,J085332.7+511849,FITS,08 53 32.751,+51 18 49.38,0.014,3.78,4.68,0.144,3.31,1.83,35.4,1,g,1,1997.32,2450566.4,0.851
9.560043,31,NGC2683,L2/S2,3.0,5.7,,,--,--,<,0.9,<,18.54,0,Simbad,NED,133.1737,33.4196,133.17205,33.421872,J085241.2+332518,FITS,08 52 41.292,+33 25 18.74,0.023,1.19,2.09,0.133,5.35,4.03,23.0,2,g,5,1994.504,2449537.6,84.129
0.892253,33,NGC2768,L2,-5.0,23.7,09 11 37.413,+60 02 14.86,0.9,7.9,,8.2,,20.73,0,Simbad,NED,137.906,60.0372,137.905862,60.037439,J091137.4+600214,FITS,09 11 37.407,+60 02 14.78,0.014,11.7,12.26,0.147,1.5,0.77,19.0,0,,1,2002.51,2452461.7,0.962
7.141463,36,NGC2841,L2,3.0,12.0,09 22 02.678,+50 58 35.70,7.4,1.1,,2.1,,19.28,0,Simbad,NED,140.5085,50.9777,140.511087,50.976567,J092202.6+505835,FITS,09 22 02.661,+50 58 35.64,0.014,1.97,3.0,0.143,5.56,1.91,18.5,1,g,1,1997.314,2450564.2,1.182
0.331187,38,NGC2911,L2,-2.0,42.2,09 33 46.108,+10 09 08.82,0.5,17.3,,17.7,,21.57,0,Simbad,NED,143.4422,10.1524,143.442112,10.152431,J093346.1+100908,FITS,09 33 46.107,+10 09 08.75,0.014,42.0,45.59,0.151,1.96,1.1,104.1,1,g,1,2000.036,2451557.8,1.635
7.12145,41,NGC3079,S2,7.0,20.4,,,--,48.0,,59.0,,21.38,7,Simbad,NED,150.4939,55.6806,150.490704,55.679783,J100157.7+554047,FITS,10 01 57.769,+55 40 47.22,0.014,157.08,293.04,0.15,5.53,4.51,169.2,1,g,6,1997.377,2450587.0,0.861


In [66]:
nvss_xmatches = XMatch.query(
    cat1=nagar_2005[0],
    cat2="vizier:VIII/65/nvss",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=10 * u.arcsec,
)

In [67]:
nvss_xmatches

angDist,recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE,_RAJ2000,_DEJ2000,NVSS,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,S1.4,e_S1.4,l_MajAxis,MajAxis,l_MinAxis,MinAxis,f_resFlux
arcsec,,,,,,hms,dms,,,,,,,,,,,,deg,deg,,"""h:m:s""","""d:m:s""",s,arcsec,mJy,mJy,,arcsec,,arcsec,
float64,int16,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float32,float64,float64,str14,str11,str11,float32,float32,float64,float32,str1,float32,str1,float32,str2
2.695816,6,NGC266,L1.9,2.0,62.4,00 49 47.819,+32 16 39.71,2.0,4.1,,4.1,,21.28,0,Simbad,NED,12.4499,32.2776,12.44942,32.27697,004947+321637,00 49 47.86,+32 16 37.1,0.29,4.0,8.0,1.1,,45.7,<,56.1,
9.129749,7,NGC315,L1.9,-4.0,65.8,,,--,470.0,,--,,23.39,9,Simbad,NED,14.4537,30.3524,14.45142,30.354,005748+302114,00 57 48.34,+30 21 14.4,0.03,0.6,772.1,25.3,,39.1,<,13.9,P*
6.759193,8,NGC404,L2,-3.0,2.4,,,--,--,<,1.3,<,17.95,0,Simbad,NED,17.3626,35.718,17.36475,35.71869,010927+354307,01 09 27.54,+35 43 07.3,0.35,5.1,3.4,0.4,<,57.0,<,44.2,
1.812751,9,NGC410,T2:,-4.0,70.6,,,--,--,<,1.0,<,20.78,4,Simbad,NED,17.746,33.152,17.74558,33.15164,011058+330905,01 10 58.94,+33 09 05.9,0.27,2.7,5.8,0.5,<,49.0,<,36.3,
2.505663,14,NGC524,T2:,-1.0,32.1,,,--,1.5,,1.5,,20.27,3,Simbad,NED,21.1992,9.5389,21.19867,9.53936,012447+093221,01 24 47.68,+09 32 21.7,0.4,5.3,3.1,0.4,<,61.5,<,49.0,
1.536308,18,NGC777,S2/L2::,-5.0,66.5,,,--,--,<,1.5,<,20.9,0,Simbad,NED,30.0625,31.4293,30.062,31.42931,020014+312545,02 00 14.88,+31 25 45.5,0.18,2.0,7.0,0.5,<,37.0,<,27.9,
6.800599,19,NGC841,L1.9:,2.1,59.5,,,--,--,<,1.5,<,20.8,0,Simbad,NED,32.8229,37.4969,32.82071,37.49764,021116+372951,02 11 16.97,+37 29 51.5,0.33,4.0,4.9,0.5,<,61.0,<,43.0,
7.250545,20,NGC1055,T2/L2::,3.0,12.6,,,--,--,<,1.8,<,19.53,0,Simbad,NED,40.4387,0.4423,40.43762,0.444,024145+002638,02 41 45.03,+00 26 38.4,0.04,0.6,198.5,6.7,,82.2,,28.5,S*


In [68]:
morx_nagar_xmatches = XMatch.query(
    cat1=nagar_2005[0],
    cat2="vizier:V/158/morxv2",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=1 * u.arcsec,
)

In [76]:
nagar_2005[0]

recno,Name,AType,TType,Dist,RAObs,DEObs,Delta,Sp15GHz,l_St15GHz,St15GHz,l_Lp15GHz,Lp15GHz,Note,Simbad,NED,_RA,_DE
,,,,Mpc,,,arcsec,mJy,,mJy,,W / Hz,,,,deg,deg
int32,str7,str9,float32,float32,str12,str12,float32,float32,str1,float32,str1,float32,int16,str6,str3,float64,float64
1,IC239,L2::,6.0,16.8,,,--,--,<,0.9,<,19.48,--,Simbad,NED,39.1163,38.9690
2,IC356,T2,2.0,18.1,,,--,--,<,1.0,<,19.59,4,Simbad,NED,61.9436,69.8126
3,IC520,T2:,2.0,47.0,,,--,--,<,1.0,<,20.42,4,Simbad,NED,133.4258,73.4908
4,IC1727,T2/L2,9.0,8.2,,,--,--,<,0.9,<,18.86,--,Simbad,NED,26.8748,27.3330
5,NGC185,S2,-5.0,0.7,,,--,--,<,1.1,<,16.81,--,Simbad,NED,9.7392,48.3373
6,NGC266,L1.9,2.0,62.4,00 49 47.819,+32 16 39.71,2.0,4.1,,4.1,,21.28,--,Simbad,NED,12.4499,32.2776
7,NGC315,L1.9,-4.0,65.8,,,--,470.0,,--,,23.39,9,Simbad,NED,14.4537,30.3524
8,NGC404,L2,-3.0,2.4,,,--,--,<,1.3,<,17.95,--,Simbad,NED,17.3626,35.7180


In [72]:
for name, _type, distance, F_15GHz in zip(
    nagar_2005[0]["Name"],
    nagar_2005[0]["AType"],
    nagar_2005[0]["Dist"],
    nagar_2005[0]["St15GHz"],
):
    if "L" in _type:
        log.info(f"considering {name}")
        sdss_id = get_source_identifier(name,"SDSS")
        fermi_id = get_source_identifier(name, "4FGL")
        distance *= u.Mpc
        
        # NVSS cross match with Nagar 2005 measurement
        this_source_nvss_xmatch = nvss_xmatches["Name"] == name
        if this_source_nvss_xmatch.any():
            nvss_xmatch_name = (
                "NVSS J" + nvss_xmatches["NVSS"][this_source_nvss_xmatch][0]
            )
            nvss_xmatch_flux = nvss_xmatches["S1.4"][this_source_nvss_xmatch][0]
            nvss_xmatch_flux_err = nvss_xmatches["e_S1.4"][this_source_nvss_xmatch][0]
        else:
            nvss_xmatch_name = ""
            nvss_xmatch_flux = 0
            nvss_xmatch_flux_err = 0

        # NVSS Cross matched with Nagar and then with MORX 
        nvss_space = insert_space_source_ids(nvss_xmatch_name)
        nvss_morx_match = nvss_space == morx[0]["NVSS-ID"]
        if nvss_morx_match.any():
            morx_nvss_name = morx[0]["NVSS-ID"][nvss_morx_match][0]
        else:
            morx_nvss_name = ""

        # FIRST measurement
        this_source_first_xmatch = first_xmatches["Name"] == name
        if this_source_first_xmatch.any():
            first_xmatch_name = (
                "FIRST " + first_xmatches["FIRST"][this_source_first_xmatch][0]
            )
            first_xmatch_flux = first_xmatches["Fint"][this_source_first_xmatch][0]
            first_xmatch_flux_err = first_xmatches["Rms"][this_source_first_xmatch][0]
        else:
            first_xmatch_name = ""
            first_xmatch_flux = 0
            first_xmatch_flux_err = 0
            
        first_morx_match = first_xmatch_name == morx[0]["FIRST-ID"]
        if first_morx_match.any():
            morx_first_name = morx[0]["FIRST-ID"][first_morx_match][0]
        else:
            morx_first_name = ""

        morx_matches = morx_nagar_xmatches["Name"] == name
        if morx_matches.any():
            morx_xmm = morx_nagar_xmatches["XMM-ID"][morx_matches]
            morx_cxo = morx_nagar_xmatches["CX-ID"][morx_matches]
            morx_swift = morx_nagar_xmatches["Swift-ID"][morx_matches]
            morx_first = morx_nagar_xmatches["FIRST-ID"][morx_matches]
            morx_nvss = morx_nagar_xmatches["NVSS-ID"][morx_matches]
            morx_lotss = morx_nagar_xmatches["LoTSS-ID"][morx_matches]
            morx_vlass = morx_nagar_xmatches["VLASS-ID"][morx_matches]
            morx_lobedist = morx_nagar_xmatches["Lobedist"][morx_matches]
        else:
            morx_xmm = ""
            morx_cxo = ""
            morx_swift = ""
            morx_first = ""
            morx_nvss = ""
            morx_lotss = ""
            morx_vlass = ""
            morx_lobedist = 0
        
        # check if the source is in the list of sources detected by Torresi et al. 2018
        torresi_detection = sdss_id in torresi_sources


        catalogue.add_row(
            [
                name,
                _type,
                sdss_id,
                fermi_id,
                nvss_xmatch_name,
                morx_nvss_name,
                first_xmatch_name,
                morx_first_name,
                morx_xmm,
                morx_cxo,
                morx_swift,
                morx_lotss,
                morx_vlass,
                morx_lobedist,
                torresi_detection,
                distance.to_value("Mpc"),
                convert_F_nu_to_luminosity(1.4 * u.GHz, nvss_xmatch_flux, u.mJy, distance),
                convert_F_nu_to_luminosity(1.4 * u.GHz, nvss_xmatch_flux_err, u.mJy, distance),
                convert_F_nu_to_luminosity(1.4 * u.GHz, first_xmatch_flux, u.mJy, distance),
                convert_F_nu_to_luminosity(1.4 * u.GHz, first_xmatch_flux_err, u.mJy, distance),
            ])

09/03/2025 09:26:11 AM|__main__|considering IC239
09/03/2025 09:26:11 AM|utils|SDSS counterpart not available for IC239
09/03/2025 09:26:11 AM|utils|4FGL counterpart not available for IC239
09/03/2025 09:26:12 AM|__main__|considering IC1727
09/03/2025 09:26:12 AM|utils|SDSS counterpart not available for IC1727
09/03/2025 09:26:12 AM|utils|4FGL counterpart not available for IC1727
09/03/2025 09:26:12 AM|__main__|considering NGC266
09/03/2025 09:26:12 AM|utils|NGC266 matched with SDSS J004947.81+321639.8 by SIMBAD
09/03/2025 09:26:12 AM|utils|4FGL counterpart not available for NGC266
09/03/2025 09:26:12 AM|__main__|considering NGC315
09/03/2025 09:26:12 AM|utils|SDSS counterpart not available for NGC315
09/03/2025 09:26:12 AM|utils|4FGL counterpart not available for NGC315
09/03/2025 09:26:12 AM|__main__|considering NGC404
09/03/2025 09:26:12 AM|utils|SDSS counterpart not available for NGC404
09/03/2025 09:26:12 AM|utils|4FGL counterpart not available for NGC404
09/03/2025 09:26:13 AM|__

In [73]:
catalogue

SOURCE_NAME,SOURCE_TYPE,SDSS-ID,FERMI-ID,NVSS-XMATCH-ID,NVSS-MORX-ID,FIRST-XMATCH-ID,FIRST-MORX-ID,XMM-MORX-ID,CXO-MORX-ID,SWIFT-MORX-ID,LoTSS-MORX-ID,VLASS-MORX,LOBE EXTENSION,DISTANCE,TORESSI DETECTION,NVSS-FLUX-XMATCH,NVSS-FLUX-ERROR-XMATCH,FIRST-FLUX-XMATCH,FIRST-FLUX-ERROR-XMATCH
,,,,,,,,,,,,,mas,Mpc,,erg / s,erg / s,erg / s,erg / s
str7,str9,str24,str17,str19,str1,str22,str1,str21,str21,str21,str22,str22,float64,float64,bool,float64,float64,float64,float64
IC239,L2::,,,,,,,,,,,,0.0,0.0,True,0.0,0.0,0.0,0.0
IC1727,T2/L2,,,,,,,,,,,,0.0,0.0,True,0.0,0.0,0.0,0.0
IC239,L2::,,,,,,,,,,,,0.0,0.0,True,0.0,0.0,0.0,0.0
IC1727,T2/L2,,,,,,,,,,,,0.0,0.0,True,0.0,0.0,0.0,0.0
IC239,L2::,,,,,,,,,,,,0.0,0.0,True,0.0,0.0,0.0,0.0
IC1727,T2/L2,,,,,,,,,,,,0.0,0.0,True,0.0,0.0,0.0,0.0
NGC266,L1.9,SDSS J004947.81+321639.8,,NVSS J004947+321637,,,,,CXOG J004947.8+321640,LSXPSJ004947.7+321640,ILTJ004947.82+321640.3,VLAJ004947.80+321639.8,0.0,0.0,True,5.217929233467116e+37,7.174652851523694e+36,0.0,0.0
NGC315,L1.9,,,NVSS J005748+302114,,,,4XMM J005748.8+302109,CXO J005748.4+302108,LSXPSJ005748.9+302107,ILDJ005748.85+302109.2,VLAJ005748.74+302110.2,47.0,0.0,True,5.599695253952314e+39,1.834895553072984e+38,0.0,0.0


In [78]:
np.array(catalogue['LOBE EXTENSION'])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0., 47.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [31]:
# let us do the same for FR0 galaxies
# we do not need to XMatch with NVSS as there is already a NVSS flux measurement for the FR0 sources
# we will not search for FIRST counterparts as there is already the NVSS flux measurement at 1.4 GHz

first_fr0_xmatches = XMatch.query(
    cat1=fr0cat[0],
    cat2="vizier:VIII/92/first14",
    colRA1="_RA",
    colDec1="_DE",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
    max_distance=10 * u.arcsec,
)

for row in fr0cat[0]:
    sdss_id = row["SimbadName"]
    log.info(f"considering {sdss_id}")
    ngc_id = get_source_identifier(sdss_id, "NGC")
    ic_id = get_source_identifier(sdss_id, "IC")
    _2mass_id = get_source_identifier(sdss_id, "2MASX")
    nvss_id = get_source_identifier(sdss_id, "NVSS")
    first_id = get_source_identifier(sdss_id, "FIRST")
    gaia_id = get_source_identifier(sdss_id, "Gaia DR3")
    fermi_id = get_source_identifier(name, "4FGL")

    distance = Distance(z=row["z"]).to("Mpc")

    L_OIII = np.power(10, row["logL[OIII]"])
    L_NVSS = np.power(10, row["logLr"])

    # FIRST measurement
    this_source_first_xmatch = first_fr0_xmatches["SimbadName"] == sdss_id
    if this_source_first_xmatch.any():
        first_xmatch_name = (
            "FIRST " + first_fr0_xmatches["FIRST"][this_source_first_xmatch][0]
        )
        first_xmatch_flux = first_fr0_xmatches["Fint"][this_source_first_xmatch][0]
        first_xmatch_flux_err = first_fr0_xmatches["Rms"][this_source_first_xmatch][0]
    else:
        first_xmatch_name = ""
        first_xmatch_flux = 0
        first_xmatch_flux_err = 0

     # XMM cross match with FR0cat measurement
        this_source_xmm_xmatch = xmm_xmatches_fr0["Name"] == name
        if this_source_xmm_xmatch.any():
            xmm_xmatch_name = (
                "4XMM " + xmm_xmatches["4XMM"][this_source_xmm_xmatch][0]
            )
            xmm_xmatch_flux = xmm_xmatches["Flux8"][this_source_xmm_xmatch][0]
            xmm_xmatch_flux_err = xmm_xmatches["e_Flux8"][this_source_xmm_xmatch][0]
        else:
            xmm_xmatch_name = ""
            xmm_xmatch_flux = 0
            xmm_xmatch_flux_err = 0

    # eventual CSC2 xmatches identified through the 2MASS or GAIA ID
    if _2mass_id:
        _2mass_xmatches = find_xmatch_id_in_catalog(
            _2mass_id,
            "2MASS21P_designation",
            csc2_xmatched,
            "CSC21P_name",
            "2MASX J",
        )

    if gaia_id:
        gaia_xmatches = find_xmatch_id_in_catalog(
            gaia_id, "GAIA21P_source_id", csc2_xmatched, "CSC21P_name", "Gaia DR3 "
        )

    # "common" (NGC or IC) name to be saved in the catalogue
    name = ngc_id if ngc_id else ic_id

    # search for an eventual swift counterpart
    # try both with the "common" name and with the 2MAS counterpart
    swift_bat_name_xmatch = find_xmatch_id_in_catalog(
        name, "CName", swift_bat_105_months[0], "Swift"
    )
    if swift_bat_name_xmatch is None:
        swift_bat_name_xmatch = find_xmatch_id_in_catalog(
            _2mass_id, "CName", swift_bat_105_months[0], "Swift"
        )

    # check if the source is in the list of sources detected by Torresi et al. 2018
    torresi_detection = sdss_id in torresi_sources

    if _2mass_id and _2mass_xmatches is not None:
        for _2mass_xmatch in _2mass_xmatches:
            coreg_fr0_expanded_catalogue.add_row(
                [
                    name,
                    "FR0",
                    sdss_id,
                    gaia_id,
                    _2mass_id,
                    nvss_id,
                    first_id,
                    "",
                    first_xmatch_name,
                    xmm_xmatch_name,
                    _2mass_xmatch["CSC21P_name"],
                    swift_name,
                    fermi_id,
                    torresi_detection,
                    distance.to_value("Mpc"),
                    L_OIII,
                    0,
                    L_NVSS,
                    0,
                    convert_F_nu_to_luminosity(
                        1.4 * u.GHz, first_xmatch_flux, u.mJy, distance
                    ),
                    convert_F_nu_to_luminosity(
                        1.4 * u.GHz, first_xmatch_flux_err, u.mJy, distance
                    ),
                    _2mass_xmatch["flux_aper_b"] * distance.to_value("cm") ** 2,
                    _2mass_xmatch["flux_aper_lolim_b"] * distance.to_value("cm") ** 2,
                    _2mass_xmatch["flux_aper_hilim_b"] * distance.to_value("cm") ** 2,
                    convert_flux_to_luminosity(xmm_xmatch_flux,'mW / m**2', distance),
                    convert_flux_to_luminosity(xmm_xmatch_flux_err,'mW / m**2', distance),
                ]
            )

    if gaia_id and gaia_xmatches is not None:
        for gaia_xmatch in gaia_xmatches:
            coreg_fr0_expanded_catalogue.add_row(
                [
                    name,
                    "FR0",
                    sdss_id,
                    gaia_id,
                    _2mass_id,
                    nvss_id,
                    first_id,
                    "",
                    first_xmatch_name,
                    xmm_xmatch_name,
                    gaia_xmatch["CSC21P_name"],
                    swift_name,
                    fermi_id,
                    torresi_detection,
                    distance.to_value("Mpc"),
                    L_OIII,
                    0,
                    L_NVSS,
                    0,
                    convert_F_nu_to_luminosity(
                        1.4 * u.GHz, first_xmatch_flux, u.mJy, distance
                    ),
                    convert_F_nu_to_luminosity(
                        1.4 * u.GHz, first_xmatch_flux_err, u.mJy, distance
                    ),
                    gaia_xmatch["flux_aper_b"] * distance.to_value("cm") ** 2,
                    gaia_xmatch["flux_aper_lolim_b"] * distance.to_value("cm") ** 2,
                    gaia_xmatch["flux_aper_hilim_b"] * distance.to_value("cm") ** 2,
                    convert_flux_to_luminosity(xmm_xmatch_flux,'mW / m**2', distance),
                    convert_flux_to_luminosity(xmm_xmatch_flux_err,'mW / m**2', distance),
                ]
            )

    else:
        coreg_fr0_expanded_catalogue.add_row(
            [
                name,
                "FR0",
                sdss_id,
                gaia_id,
                _2mass_id,
                nvss_id,
                first_id,
                "",
                first_xmatch_name,
                xmm_xmatch_name,
                "",
                swift_name,
                fermi_id,
                torresi_detection,
                distance.to_value("Mpc"),
                L_OIII,
                0,
                L_NVSS,
                0,
                convert_F_nu_to_luminosity(
                    1.4 * u.GHz, first_xmatch_flux, u.mJy, distance
                ),
                convert_F_nu_to_luminosity(
                    1.4 * u.GHz, first_xmatch_flux_err, u.mJy, distance
                ),
                0,
                0,
                0,
                convert_flux_to_luminosity(xmm_xmatch_flux,'mW / m**2', distance),
                convert_flux_to_luminosity(xmm_xmatch_flux_err,'mW / m**2', distance),
            ]
        )

08/14/2025 03:37:09 PM|__main__|considering SDSS J010852.48-003919.4
08/14/2025 03:37:09 PM|utils|NGC counterpart not available for SDSS J010852.48-003919.4
08/14/2025 03:37:09 PM|utils|IC counterpart not available for SDSS J010852.48-003919.4
08/14/2025 03:37:09 PM|utils|SDSS J010852.48-003919.4 matched with 2MASX J01085249-0039187 by SIMBAD
08/14/2025 03:37:09 PM|utils|SDSS J010852.48-003919.4 matched with NVSS J010852-003922 by SIMBAD
08/14/2025 03:37:09 PM|utils|SDSS J010852.48-003919.4 matched with FIRST J010852.4-003919 by SIMBAD
08/14/2025 03:37:09 PM|utils|SDSS J010852.48-003919.4 matched with Gaia DR3 2533177556632120064 by SIMBAD
08/14/2025 03:37:09 PM|utils|4FGL counterpart not available for NGC7814
08/14/2025 03:37:09 PM|__main__|2MASX J01085249-0039187 not matched in the catalogue
08/14/2025 03:37:09 PM|__main__|Gaia DR3 2533177556632120064 not matched in the catalogue
08/14/2025 03:37:09 PM|__main__| not matched in the catalogue
08/14/2025 03:37:09 PM|__main__|2MASX J0108

In [32]:
coreg_fr0_expanded_catalogue

SOURCE_NAME,SOURCE_TYPE,SIMBAD_SDSS_ID,SIMBAD_GAIA_DR3_ID,SIMBAD_2MASS_ID,SIMBAD_NVSS_ID,SIMBAD_FIRST_ID,XMATCH_NVSS_ID,XMATCH_FIRST_ID,XMATCH_XMM_ID,2CXO_ID,SWIFT_ID,SIMBAD_4FGL_ID,TORRESI_2016_SAMPLE,DISTANCE,L_OIII,L_15GHz,L_1400MHz_NVSS,L_1400MHz_NVSS_ERR,L_1400MHz_FIRST,L_1400MHz_FIRST_ERR,L_500_7000eV_CHANDRA,L_500_7000eV_CHANDRA_LOLIM,L_500_7000eV_CHANDRA_HILIM,Mean_flux_200-12000eV_XMM,e_Mean_flux_200-12000eV_XMM
,,,,,,,,,,,,,,Mpc,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s,erg / s
str10,str9,str24,str28,str23,str19,str22,str19,str22,str21,str21,str18,str17,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC239,L2::,,Gaia DR3 334974257582526336,2MASX J02362783+3858085,,,,,,,,,False,16.799999237060547,0.0,4.5589330341014714e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IC1727,T2/L2,,,2MASXI J0147306+271952,,,,,,,,,False,8.199999809265137,0.0,1.0861064740146649e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NGC266,L1.9,SDSS J004947.81+321639.8,Gaia DR3 360950670758689280,2MASX J00494779+3216398,NVSS J004947+321637,,NVSS J004947+321637,,,,,,False,62.400001525878906,0.0,2.865202146374286e+38,5.217929233467116e+37,7.174652851523694e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NGC315,L1.9,,Gaia DR3 311721858691625984,2MASX J00574891+3021083,NVSS J005748+302114,,NVSS J005748+302114,,4XMM J005748.8+302109,,,,False,65.80000305175781,0.0,0.0,5.599695253952314e+39,1.834895553072984e+38,0.0,0.0,0.0,0.0,0.0,6.510460013791378e+41,8.073488456489715e+39
NGC404,L2,,,2MASX J01092707+3543046,,,NVSS J010927+354307,,,,,,False,2.4000000953674316,0.0,1.3439033935001284e+35,3.2805027629233096e+34,3.859415072713659e+33,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NGC428,L2/T2:,SDSS J011255.76+005853.4,,2MASX J01125570+0058536,,,,,,,,,False,14.899999618530273,0.0,3.586057406164185e+36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NGC474,L2::,,Gaia DR3 2562761428805166080,2MASX J01200669+0324549,,,,,4XMM J012006.7+032456,,,,False,32.5,0.0,2.843545583024052e+37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0330268226964055e+39,4.8546273991423515e+38
NGC718,L2,,Gaia DR3 2512927232507396608,2MASX J01531331+0411453,,,,,,,,,False,21.399999618530273,0.0,1.2328805783704804e+37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
np.array(coreg_fr0_expanded_catalogue['XMATCH_XMM_ID'][-109:-1])

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', ''], dtype='<U21')

In [34]:
path = Path("./")
path.mkdir(exist_ok=True, parents=True)
coreg_fr0_expanded_catalogue.write(
    path / "coreg_fr0_expanded_catalogue.fits", overwrite=True
)